# 사전 작업

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 작업 디렉토리 변경
import os
os.chdir('/content/drive/MyDrive/U-Sto_AI') # 파일이 위치한 폴더 경로

# 라이브러리 설치
!pip install openai langchain

In [ ]:
# OPENAI 연결 확인용 테스트
from google.colab import userdata
from openai import OpenAI

# 저장해둔 API 키 불러오기
api_key = userdata.get('OPENAI_API_KEY')

# 클라이언트 생성
client = OpenAI(api_key=api_key)

# 테스트 요청 (연결 확인용)
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": "안녕, 연결 테스트 중이야."}]
    )
    print("OpenAI 연결 성공!")
    print("답변:", response.choices[0].message.content)
except Exception as e:
    print("연결 실패:", e)

# 1. 시스템 페르소나 및 프롬프트 정의

In [ ]:
# [프롬프트] 시스템 페르소나 정의 (SCM 전문가)
SCM_SYSTEM_PROMPT = """
당신은 SCM(공급망 관리) 전문 AI 어시스턴트입니다.
사용자의 질문을 분석하여 다음 두 가지 핵심 정보를 추출하는 것이 목표입니다.

1. **대상 품목 (Target Item)**: 사용자가 언급한 물품명 (예: 노트북, 모니터 등). 언급이 없으면 null.
2. **재고 관리 리스크 성향 (Risk Attitude)**: 사용자의 의도를 파악하여 '서비스 계수(Z값)'를 매핑.

### [Z값 매핑 가이드라인]
1. **보수적 운영 (Conservative)**:
   - 키워드: "절대 결품 안 됨", "수업 지장 없이", "매우 중요", "넉넉하게", "여유 있게", "평소대로".
   - **Z값: 1.65** (서비스 수준 약 95% 확보 - 안전 제일)

2. **공격적 운영 (Aggressive)**:
   - 키워드: "재고 최소화", "예산 삭감", "공간 부족", "타이트하게", "비용 절감", "0으로".
   - **Z값: 0.0** (안전버퍼 없음 - 비용 절감 최우선)

### [출력 형식 (JSON Only)]
반드시 아래 키를 포함한 JSON 형식으로만 응답하세요:
{
  "target_item": "추출된 물품명 또는 null",
  "intent_category": "Conservative" | "Aggressive",
  "z_value": 1.65 | 0.0,
  "reasoning": "선택 근거 요약 (한국어)"
}
"""

In [ ]:
import json
import os
from openai import OpenAI
from google.colab import userdata

# [설정] Colab Secrets에서 API 키 가져오기
try:
    # Colab 왼쪽 '열쇠 모양'에 저장된 이름(OPENAI_API_KEY)을 호출
    api_key = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
    print("API Key 로드 성공! OpenAI 클라이언트가 연결되었습니다.")
except Exception as e:
    print("API Key 로드 실패! 열쇠 모양(Secrets)에 저장된 이름이 맞는지 확인해주세요.")
    print(f"에러 메시지: {e}")

# [함수] LLM 호출 및 파라미터 추출
def extract_query_parameters(user_query, model="gpt-4o"):
    """
    사용자의 질문(Query)을 받아 Z값과 타겟 품목을 JSON으로 반환하는 함수
    """
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": SCM_SYSTEM_PROMPT},
                {"role": "user", "content": user_query}
            ],
            response_format={"type": "json_object"},
            temperature=0.0 # 분석의 일관성을 위해 0으로 설정
        )

        # 응답 파싱
        result = json.loads(response.choices[0].message.content)

        # [방어 로직] 키 누락 시 기본값 처리 (안전하게 Conservative 1.65 적용)
        if 'z_value' not in result: result['z_value'] = 1.65
        if 'target_item' not in result: result['target_item'] = None

        return result

    except Exception as e:
        print(f"Error analyzing query: {e}")
        # 에러 발생 시 안전한 기본값 반환
        return {
            "target_item": None,
            "intent_category": "Conservative (Fallback)",
            "z_value": 1.65,
            "reasoning": "시스템 에러로 인한 기본값 적용"
        }

In [ ]:
# [테스트] 실행 결과 확인
if __name__ == "__main__":
    scenarios = [
        "이번 학기 노트북은 수업 중에 고장 나면 큰일 나니까 무조건 넉넉하게 잡아줘.",
        "올해 예산이 너무 삭감됐어. 모니터 재고 비용을 0으로 만들고 싶어.",
        "다기능복사기는 그냥 평소대로 운영해."
    ]

    print("-" * 110)
    print(f"{'Item':<10} | {'Intent':<12} | {'Z-Value':<8} | {'Reasoning'}")
    print("-" * 110)

    for query in scenarios:
        result = extract_query_parameters(query)

        item_disp = result['target_item'] if result['target_item'] else "전체"
        print(f"{item_disp:<10} | {result['intent_category']:<12} | {str(result['z_value']):<8} | {result['reasoning']}")

# 2. 안전버퍼 산출 함수 구현

In [ ]:
import math
from datetime import datetime, timedelta

def get_lead_time_days(grade):
    """
    [DB 연동] 리드타임 등급을 실제 일수로 변환
    Grade 0: 1주 (7일) / Grade 1: 3주 (20일) / Grade 2: 6주 (38일)
    """
    mapping = {0: 7, 1: 20, 2: 38}
    return mapping.get(grade, 7) # 기본값 7일

def calculate_safety_metrics(z_value, model_rmse, lead_time_grade, predicted_failure_date_str):
    """
    Z값과 모델 오차(RMSE)를 이용해 '안전버퍼(일수)'를 계산하고,
    최종 '권장 발주일(D-Day)'을 산출하는 함수
    """
    # 1. 날짜 포맷 변환 (String -> Datetime)
    try:
        pred_date = datetime.strptime(predicted_failure_date_str, "%Y-%m-%d")
    except ValueError:
        return {"error": "Invalid date format"}

    # 2. 리드타임(조달 기간) 가져오기
    lt_days = get_lead_time_days(lead_time_grade)

    # 3. 안전버퍼(Safety Buffer) 계산 [핵심 로직]
    # 공식: Buffer(일) = Z * RMSE(일)
    # 소수점은 올림(ceil) 처리하여 안전하게 하루라도 더 확보
    buffer_days = math.ceil(z_value * model_rmse)

    # 4. 총 준비 기간 (리드타임 + 안전버퍼)
    total_prep_days = lt_days + buffer_days

    # 5. 권장 발주일(Reorder Point) 계산
    # 예측된 고장일로부터 준비 기간만큼 거꾸로 계산
    recommend_date = pred_date - timedelta(days=total_prep_days)

    # 6. 결과 리포트 생성
    return {
        "z_used": z_value,
        "rmse_used": model_rmse,
        "buffer_days": buffer_days,      # AI가 추가한 안전 기간
        "lead_time_days": lt_days,       # 물건 오는데 걸리는 시간
        "total_prep_days": total_prep_days,
        "predicted_failure": predicted_failure_date_str,
        "recommend_order_date": recommend_date.strftime("%Y-%m-%d")
    }

In [ ]:
# [테스트] 시나리오별 결과 확인
if __name__ == "__main__":
    # 가정: 하이브리드 모델이 예측한 오차(RMSE)가 '5일' 정도임
    current_model_rmse = 5.0

    # 가정: AI가 예측한 이 장비의 고장 날짜
    ai_predicted_date = "2026-03-20"

    # 가정: 이 장비(노트북)는 해외 배송이라 리드타임 등급이 '1(20일)'임
    item_lead_time_grade = 1

    # 1단계에서 확정한 Z값 적용 (Conservative: 1.65 / Standard: 0.84 / Aggressive: 0.0)
    scenarios = [
        {"intent": "보수적 (1.65)", "z": 1.65},
        {"intent": "공격적 (0.0)",  "z": 0.0}
    ]

    print(f"■ 모델 오차(RMSE): {current_model_rmse}일 / 예측 고장일: {ai_predicted_date} / 리드타임: 20일")
    print("-" * 110)
    print(f"{'성향(Intent)':<15} | {'Z값':<5} | {'안전버퍼(일)':<12} | {'총 준비기간':<12} | {'★ 권장 발주일 (Order Date)'}")
    print("-" * 110)

    for case in scenarios:
        result = calculate_safety_metrics(
            z_value=case['z'],
            model_rmse=current_model_rmse,
            lead_time_grade=item_lead_time_grade,
            predicted_failure_date_str=ai_predicted_date
        )

        print(f"{case['intent']:<15} | {result['z_used']:<5} | {str(result['buffer_days']) + '일':<12} | {str(result['total_prep_days']) + '일':<12} | {result['recommend_order_date']}")

# 3. 인사이트 생성 및 리포트 포맷팅

In [ ]:
import json
import os
from openai import OpenAI
from google.colab import userdata

# API 키 로드 (기존 방식 유지)
try:
    api_key = userdata.get('OPENAI_API_KEY')
    client = OpenAI(api_key=api_key)
except Exception as e:
    print(f"API Key 설정 확인 필요: {e}")

# 3. 인사이트 생성 및 리포트 작성 프롬프트 (수정됨: 시간/날짜 중심)
REPORT_SYSTEM_PROMPT = """
당신은 대학 자산 관리 실무자(User)를 돕는 'SCM AI 분석 파트너'입니다.
AI가 계산한 **'발주 시점 데이터'**를 바탕으로, 실무자가 **'언제 주문해야 하는지'** 명확한 행동 지침을 제공하세요.

[입력 데이터]
1. 품목명: 관리 대상 (예: 노트북)
2. 전략 (Z값): 사용자가 선택한 리스크 성향 (보수적일수록 발주 시점을 당김)
3. 산출된 안전버퍼(일수): 예측 오차를 고려하여 며칠을 더 미리 주문하는지
4. 권장 발주일(D-Day): AI가 계산한 최종 주문 데드라인

[작성 가이드]
- **독자:** 실제 발주를 넣는 '자산 관리 담당자'
- **톤앤매너:** 명확하고 정중한 '해요체'. (예: "안전버퍼를 9일 확보하여 2월 19일까지 발주하시길 권장합니다.")
- **핵심 내용:**
  - 사용자의 성향(Z값) 때문에 안전기간을 얼마나 더 잡았는지 설명.
  - **'권장 발주일'**을 강조하여 결품(Go-Dark)을 막도록 유도.

[출력 형식 (JSON)]
반드시 아래 키를 포함한 **JSON 형식**으로만 응답하세요:
{
  "report_title": "직관적인 제목 (예: [D-7] 노트북 발주 시점 도래 알림)",
  "analysis_summary": "상황 분석 (예측 오차와 리스크 성향을 고려한 버퍼 설명)",
  "action_item": "구체적 실행 가이드 (날짜 포함)",
  "alert_level": "Normal" | "High" | "Critical"
}
"""

def generate_scm_report(item_name, strategy_info, step2_result):
    """
    [Step 3] 계산된 결과(Step 2)를 받아서 리포트 문구(Text)를 생성하는 함수
    """
    # LLM에게 던져줄 상황 요약 정보 (시간 관점으로 변경)
    context_input = f"""
    - 품목명: {item_name}
    - 사용자의 전략: {strategy_info['intent_category']} (Z값: {strategy_info['z_value']})
    - 모델 오차(RMSE): {step2_result['rmse_used']}일 (이만큼 날짜가 틀릴 수 있음)
    - 확보한 안전버퍼: {step2_result['buffer_days']}일 (오차를 커버하기 위해 앞당긴 기간)
    - 총 조달 기간(리드타임 포함): {step2_result['total_prep_days']}일
    - ★ AI 권장 발주 날짜: {step2_result['recommend_order_date']}
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini", # 가성비 모델 사용
            messages=[
                {"role": "system", "content": REPORT_SYSTEM_PROMPT},
                {"role": "user", "content": context_input}
            ],
            response_format={"type": "json_object"},
            temperature=0.7 # 문장 생성이므로 약간의 창의성 허용
        )
        return json.loads(response.choices[0].message.content)
    except Exception as e:
        print(f"리포트 생성 중 에러 발생: {e}")
        return None

In [ ]:
# [테스트] Step 1, 2, 3 전체 연결 시뮬레이션
if __name__ == "__main__":
    print("🚀 [Step 3] AI 리포트 생성 테스트 시작...\n")

    # [가정] Step 1 결과 (사용자: 보수적)
    mock_step1_strategy = {"intent_category": "Conservative", "z_value": 1.65}

    # [가정] Step 2 결과 (계산된 날짜들)
    # 아까 2단계 코드에서 나온 결과를 그대로 흉내 냄
    mock_step2_result = {
        "rmse_used": 5.0,
        "buffer_days": 9,             # 1.65 * 5.0 = 8.25 -> 9일 올림
        "total_prep_days": 29,        # 리드타임 20 + 버퍼 9
        "recommend_order_date": "2026-02-19" # 오늘보다 미래라고 가정
    }

    # [실행] 리포트 생성
    final_report = generate_scm_report(
        item_name="노트북(교육용)",
        strategy_info=mock_step1_strategy,
        step2_result=mock_step2_result
    )

    # [결과 출력]
    if final_report:
        print(f"📄 제목: {final_report['report_title']}")
        print(f"📊 분석: {final_report['analysis_summary']}")
        print(f"✅ 조치: {final_report['action_item']}")
        print(f"🚨 레벨: {final_report['alert_level']}")
    else:
        print("리포트 생성 실패")

# 4. 시나리오 테스트 및 정합성 검증

In [ ]:
import math
import time

# 통합 시뮬레이션 및 검증 함수
def run_full_simulation(user_query, item_name, model_rmse, lead_time_grade, predicted_date):
    print("\n" + "=" * 80)
    print(f"🎬 [시나리오 테스트] 사용자 질문: \"{user_query}\"")
    print("=" * 80)

    # 1단계: 사용자 의도 파악 (Intent Recognition)
    print("\n▶ 1단계: 의도 및 Z값 추출 중...")
    try:
        # 1단계 함수 호출
        step1_result = extract_query_parameters(user_query)

        # 결과 출력
        z_val = step1_result['z_value']
        intent = step1_result['intent_category']
        print(f"   ㄴ 결과: 성향={intent}, Z값={z_val}")

    except Exception as e:
        print(f"!! 1단계 에러: {e}")
        return

    # 2단계: 안전버퍼 및 발주일 계산 (Calculation)
    print(f"\n▶ 2단계: 안전버퍼(일수) 계산 중... (RMSE: {model_rmse}일)")
    try:
        # 2단계 함수 호출
        step2_result = calculate_safety_metrics(
            z_value=z_val,
            model_rmse=model_rmse,
            lead_time_grade=lead_time_grade,
            predicted_failure_date_str=predicted_date
        )

        # 결과 출력
        buffer_days = step2_result['buffer_days']
        rec_date = step2_result['recommend_order_date']
        print(f"   ㄴ 결과: 안전버퍼={buffer_days}일, 권장발주일={rec_date}")

    except Exception as e:
        print(f"!! 2단계 에러: {e}")
        return

    # [검증 포인트] Python 계산이 정확한지 기계적으로 체크 (Assert)
    expected_buffer = math.ceil(z_val * model_rmse)
    if buffer_days == expected_buffer:
        print(" [Pass] 데이터 정합성 검증 성공 (Python 계산 일치)")
    else:
        print(f"[Fail] 데이터 불일치! (계산값: {expected_buffer} vs 함수반환값: {buffer_days})")

    # 3단계: 리포트 생성 (Report Generation)
    print("\n▶ 3단계: 최종 리포트 생성 중...")
    try:
        # 3단계 함수 호출 (2단계 결과를 입력으로 사용)
        final_report = generate_scm_report(item_name, step1_result, step2_result)

        if final_report:
            print("\n" + "■" * 25 + " [최종 AI 리포트] " + "■" * 25)
            print(f"📄 제목:  {final_report.get('report_title')}")
            print(f"📊 분석:  {final_report.get('analysis_summary')}")
            print(f"✅ 조치:  {final_report.get('action_item')}")
            print(f"🚨 레벨:  {final_report.get('alert_level')}")
            print("-" * 70)
        else:
            print("!! 3단계 리포트 생성 실패")

    except Exception as e:
        print(f"!! 3단계 에러: {e}")

In [ ]:
# [실행] 가상 시나리오 테스트
if __name__ == "__main__":
    # 변수 설정
    target_item = "고성능 워크스테이션"
    mock_rmse = 4.5
    mock_lt_grade = 1
    mock_pred_date = "2026-03-15"

    # 질문 설정
    input_query = "이번 학기 컴퓨터공학과 실습실 장비 교체해야 해? 수업 지장 없게 넉넉하게 잡아줘."

    # 실행
    run_full_simulation(input_query, target_item, mock_rmse, mock_lt_grade, mock_pred_date)